## learning kernels for the GLM example. 



In [ ]:
import delfi.distribution as dd
import delfi.generator as dg
import delfi.inference as infer
import delfi.utils.io as io
import delfi.summarystats as ds
import lfimodels.glm.utils as utils
import matplotlib.pyplot as plt
import numpy as np

from lfimodels.glm.GLM import GLM
from lfimodels.glm.GLMStats import GLMStats
from delfi.utils.viz import plot_pdf

%matplotlib inline

data_seed = 99
seed = 42
duration = 300

len_filter, a = 9, 1.
true_params, labels_params = utils.obs_params(len_filter, a)

obs = utils.obs_data(true_params, seed=data_seed, duration=duration)
obs_stats = utils.obs_stats(true_params, seed=data_seed, duration=duration)

rerun = False  # if False, will try loading file from disk

def init_g():
    
    s = GLMStats(n_summary=len_filter+1)

    p = utils.smoothing_prior(n_params=len_filter+1, seed=seed)
    m = GLM(len_filter=len_filter, duration=duration, seed=data_seed)
    g = dg.Default(summary=s, model=m, prior=p)    
    
    #g = dg.MPGenerator(models=[GLM(len_filter=len_filter, duration=duration, seed=seed+i) for i in range(n_workers)], 
    #                   prior=p, summary=s)
    
    return g

try:
    assert rerun == False, 'rerun requested'
    sam = np.load('sam.npz')['arr_0']
except:
    sam = utils.pg_mcmc(true_params, obs, duration=duration, 
                        prior_dist=utils.smoothing_prior(n_params=len_filter+1, seed=data_seed))
    np.savez('sam.npz', sam)
pe = dd.Gaussian(m=sam.mean(axis=1), S=np.cov(sam))
    
n_train=10000
n_rounds=5
minibatch=100 
epochs=200   
round_cl=3

minibatch_cbk=100
epochs_cbk=500

n_hiddens = [50,50]
convert_to_T = 3
pilot_samples = 1000
svi=True
reg_lambda=0.01
prior_norm=False
n_workers = 10    

res = infer.SNPE(generator=init_g(), 
                 obs=obs_stats, 
                 n_hiddens=n_hiddens, 
                 seed=seed, 
                 convert_to_T=convert_to_T, 
                 pilot_samples=pilot_samples,
                 svi=svi,
                 reg_lambda=reg_lambda,
                 prior_norm=prior_norm)

logs, tds, posteriors = res.run(n_train=n_train, 
                                n_rounds=n_rounds, 
                                minibatch=minibatch, 
                                epochs=epochs, 
                                minibatch_cbk=minibatch_cbk,
                                epochs_cbk=epochs_cbk,                                        
                                round_cl=round_cl, 
                                kernel_loss=None)

# version with learned kernels

## x_kl loss

In [ ]:


res_k = infer.SNPE(generator=init_g(), 
                 obs=obs_stats, 
                 n_hiddens=n_hiddens, 
                 seed=seed, 
                 convert_to_T=convert_to_T, 
                 pilot_samples=pilot_samples,
                 svi=svi,
                 reg_lambda=reg_lambda,
                 prior_norm=prior_norm)

logs_k, tds_k, posteriors_k = res_k.run(n_train=n_train, 
                                n_rounds=n_rounds, 
                                minibatch=minibatch, 
                                epochs=epochs, 
                                minibatch_cbk=minibatch_cbk,
                                epochs_cbk=epochs_cbk,                                        
                                round_cl=round_cl, 
                                cbk_feature_layer=4,
                                kernel_loss='x_kl')

In [ ]:
obs_statz = (obs_stats - res.stats_mean) / res.stats_std 

def d_kl_gauss(m1, m2, S1, S2):
   
    S2i = np.linalg.inv(S2)
    dm =  m2 - m1
   
    out  = np.trace(S2i.dot(S1))
    out += dm.dot(S2i.dot(dm))
    out -= m1.size
    out += np.prod(np.linalg.slogdet(S2)) - np.prod(np.linalg.slogdet(S1))
    return out / 2.

# run with Gaussian proposals
for r in range(len(tds_k)):
    plot_pdf(posteriors[r],
             pdf2=posteriors_k[r], 
             lims=[-2,2], 
             samples=sam, 
             gt=true_params, 
             resolution=100,
             figsize=(16,16));


    stats, w = tds[r][1], tds[r][2]
    w /= w.sum()
    px = dd.Gaussian(m=w.dot(stats),
                     S=np.cov(stats.T, aweights=w))
    stats, w = tds_k[r][1], tds_k[r][2]
    w /= w.sum()
    px_k = dd.Gaussian(m=w.dot(stats),
                     S=np.cov(stats.T, aweights=w))
    stats = tds_k[r][1]
    plot_pdf(px, pdf2=px_k, samples=stats.T, lims = [-2, 2], resolution=100,
             gt=obs_statz.reshape(-1,1), figsize=(16,16));    
    
    
    pdf1, pdf2  = posteriors[r].xs[0], posteriors_k[r].xs[0]
    m1, m2, S1, S2 = pdf1.m, pdf2.m, pdf1.S, pdf2.S
    dr = d_kl_gauss(pe.m, m1, pe.S, S1)
    dk = d_kl_gauss(pe.m, m2, pe.S, S2)
    print('D_KL: ', (dr, dk))
    
    iws = tds[r][2]
    iws /= iws.sum()

    kiws = tds_k[r][2]
    kiws /= kiws.sum()
    print('ESS: ', 1/np.sum(iws**2), 1/np.sum(kiws**2))
    

In [ ]:


res_k = infer.SNPE(generator=init_g(), 
                 obs=obs_stats, 
                 n_hiddens=n_hiddens, 
                 seed=seed, 
                 convert_to_T=convert_to_T, 
                 pilot_samples=pilot_samples,
                 svi=svi,
                 reg_lambda=reg_lambda,
                 prior_norm=prior_norm)

logs_k, tds_k, posteriors_k = res_k.run(n_train=n_train, 
                                n_rounds=n_rounds, 
                                minibatch=minibatch, 
                                epochs=epochs, 
                                minibatch_cbk=minibatch_cbk,
                                epochs_cbk=epochs_cbk,                                        
                                round_cl=round_cl, 
                                cbk_feature_layer=0,
                                kernel_loss='x_kl')

In [ ]:
obs_statz = (obs_stats - res.stats_mean) / res.stats_std 

def d_kl_gauss(m1, m2, S1, S2):
   
    S2i = np.linalg.inv(S2)
    dm =  m2 - m1
   
    out  = np.trace(S2i.dot(S1))
    out += dm.dot(S2i.dot(dm))
    out -= m1.size
    out += np.prod(np.linalg.slogdet(S2)) - np.prod(np.linalg.slogdet(S1))
    return out / 2.

# run with Gaussian proposals
for r in range(len(tds_k)):
    plot_pdf(posteriors[r],
             pdf2=posteriors_k[r], 
             lims=[-2,2], 
             samples=sam, 
             gt=true_params, 
             resolution=100,
             figsize=(16,16));


    stats, w = tds[r][1], tds[r][2]
    w /= w.sum()
    px = dd.Gaussian(m=w.dot(stats),
                     S=np.cov(stats.T, aweights=w))
    stats, w = tds_k[r][1], tds_k[r][2]
    w /= w.sum()
    px_k = dd.Gaussian(m=w.dot(stats),
                     S=np.cov(stats.T, aweights=w))
    stats = tds_k[r][1]
    plot_pdf(px, pdf2=px_k, samples=stats.T, lims = [-2, 2], resolution=100,
             gt=obs_statz.reshape(-1,1), figsize=(16,16));    
    
    
    pdf1, pdf2  = posteriors[r].xs[0], posteriors_k[r].xs[0]
    m1, m2, S1, S2 = pdf1.m, pdf2.m, pdf1.S, pdf2.S
    dr = d_kl_gauss(pe.m, m1, pe.S, S1)
    dk = d_kl_gauss(pe.m, m2, pe.S, S2)
    print('D_KL: ', (dr, dk))
    
    iws = tds[r][2]
    iws /= iws.sum()

    kiws = tds_k[r][2]
    kiws /= kiws.sum()
    print('ESS: ', 1/np.sum(iws**2), 1/np.sum(kiws**2))
    

In [ ]:
plt.figure(figsize=(12,8))
for r in range(n_rounds):
    plt.subplot(n_rounds,3,3*r+1)
    plt.plot(logs[r]['loss'])
    plt.subplot(n_rounds,3,3*r+2)
    plt.plot(logs_k[r]['loss'])    
    plt.subplot(n_rounds,3,3*r+3)
    if r > 0:
        plt.plot(logs_k[r]['cbk_loss'])    
plt.show()

In [ ]:
from delfi.kernel.Kernel_learning import kernel_opt
import lasagne.layers as ll
import lasagne.updates as lu
import theano 


kernel_loss = 'x_kl'
trn_data = tds_k[1]
n_train_round = n_train
iws = res_k.generator.prior.eval(trn_data[0], log=False) / posteriors_k[0].eval(trn_data[0], log=False)

ks = list(res_k.network.layer.keys())
layer_index = 0
hl = res_k.network.layer[ks[layer_index]]

plt.figure(figsize=(12,10))
clrs = ['k', 'r']

iws_  = iws/iws.sum()
print('ESS pre: ', 1./sum(iws_**2))
for z in range(2):
    
    #zscore = False if z==0 else True
    #epochs__ = 2000
    #minibatch__ = n_train
    
    zscore=True
    if z==1:
        epochs__ = 2000
        minibatch__ = n_train
    elif z==0:
        epochs__ = 2000
        minibatch__=minibatch 

    
    print('condition :' + str(z))
    print('minibatch =' + str(minibatch__))
    print('epochs =' + str(epochs__))
    stats = trn_data[1].reshape(n_train_round,-1).copy()
    obs_z = obs_stats.copy()
    if zscore: 
        obs_z = ( obs_z - res_k.stats_mean) / res_k.stats_std
        #stats = ( stats - stats.mean(axis=0)) / stats.std(axis=0)
    stat_features = theano.function(
        inputs=[res_k.network.stats],
        outputs=ll.get_output(hl))
    fstats = stat_features(stats)
    fobs_z = stat_features(obs_z)

    
    for r in range(10):
        cbkrnl, cbl = kernel_opt(
            iws=iws.astype(np.float32), 
            stats=fstats,
            obs=fobs_z, 
            kernel_loss=kernel_loss, 
            epochs=epochs__, 
            minibatch=minibatch__, 
            stop_on_nan=False,
            seed=res.gen_newseed(), 
            step=lu.adam,
            monitor=res.monitor_dict_from_names(None))

        kiws = iws * cbkrnl.eval(fstats)

        kiws_ = kiws/kiws.sum()
        print('ESS post:', 1./sum(kiws_**2))

        plt.subplot(1,3,1)
        plt.plot(cbl, color=clrs[z])
        plt.title('final loss' + str(cbl[-1]))
        #plt.subplot(1,3,2)
        #plt.plot(cbkrnl.A / cbkrnl.A.sum(), color=clrs[z])
        plt.subplot(1,3,2)
        if zscore:
            plt.plot(cbkrnl.A / trn_data[1].reshape(n_train_round,-1).copy().var(axis=0), color=clrs[z])
        else:
            plt.plot(cbkrnl.A , color=clrs[z])
        plt.subplot(1,3,3)
        if z==0:
            x_ = cbkrnl.eval(fstats)
        elif z==1:
            y_ = cbkrnl.eval(fstats)
            plt.plot(x_, y_, '.')
plt.show()


## mESS loss

In [ ]:
m = GLM(duration=duration, seed=seed)
p = utils.smoothing_prior(n_params=m.n_params, seed=seed)
s = GLMStats(n_summary=m.n_params)
g = dg.Default(model=m, prior=p, summary=s)
res_k2 = infer.SNPE(generator=init_g(), 
                 obs=obs_stats, 
                 n_hiddens=n_hiddens, 
                 seed=seed, 
                 convert_to_T=convert_to_T, 
                 pilot_samples=pilot_samples,
                 svi=svi,
                 reg_lambda=reg_lambda,
                 prior_norm=prior_norm)

logs_k2, tds_k2, posteriors_k2 = res_k2.run(n_train=n_train, 
                                n_rounds=n_rounds, 
                                minibatch=minibatch, 
                                epochs=epochs, 
                                minibatch_cbk=minibatch_cbk,
                                epochs_cbk=epochs_cbk,                                        
                                round_cl=round_cl, 
                                kernel_loss='ess')

iws = tds[1][2]
iws /= iws.sum()

kiws = tds_k2[1][2]
kiws /= kiws.sum()
1/np.sum(iws**2), 1/np.sum(kiws**2)

In [ ]:
from delfi.kernel.Kernel_learning import kernel_opt
import lasagne.layers as ll
import lasagne.updates as lu
import theano 


kernel_loss = 'ess'
trn_data = tds_k[1]
n_train_round = n_train
iws = res_k.generator.prior.eval(trn_data[0], log=False) / posteriors_k[0].eval(trn_data[0], log=False)

ks = list(res_k.network.layer.keys())
layer_index = 0
hl = res_k.network.layer[ks[layer_index]]

plt.figure(figsize=(12,10))
clrs = ['k', 'r']

iws_  = iws/iws.sum()
print('ESS pre: ', 1./sum(iws_**2))
for z in range(2):
    
    #zscore = False if z==0 else True
    #epochs__ = 2000
    #minibatch__ = n_train
    
    zscore=True
    if z==1:
        epochs__ = 2000
        minibatch__ = n_train
    elif z==0:
        epochs__ = 2000
        minibatch__=minibatch 

    
    print('zscoring =' + str(z))
    print('minibatch =' + str(minibatch__))
    print('epochs =' + str(epochs__))
    stats = trn_data[1].reshape(n_train_round,-1).copy()
    obs_z = obs_stats.copy()
    if zscore: 
        obs_z = ( obs_z - res_k.stats_mean) / res_k.stats_std
        #stats = ( stats - stats.mean(axis=0)) / stats.std(axis=0)
    stat_features = theano.function(
        inputs=[res_k.network.stats],
        outputs=ll.get_output(hl))
    fstats = stat_features(stats)
    fobs_z = stat_features(obs_z)

    
    for r in range(10):
        cbkrnl, cbl = kernel_opt(
            iws=iws.astype(np.float32), 
            stats=fstats,
            obs=fobs_z, 
            kernel_loss=kernel_loss, 
            epochs=epochs__, 
            minibatch=minibatch__, 
            stop_on_nan=False,
            seed=res.gen_newseed(), 
            step=lu.adam,
            monitor=res.monitor_dict_from_names(None))

        kiws = iws * cbkrnl.eval(fstats)

        kiws_ = kiws/kiws.sum()
        print('ESS post:', 1./sum(kiws_**2))

        plt.subplot(1,3,1)
        plt.plot(cbl, color=clrs[z])
        plt.title('final loss' + str(cbl[-1]))
        #plt.subplot(1,3,2)
        #plt.plot(cbkrnl.A / cbkrnl.A.sum(), color=clrs[z])
        plt.subplot(1,3,2)
        if zscore:
            plt.plot(cbkrnl.A / trn_data[1].reshape(n_train_round,-1).copy().var(axis=0), color=clrs[z])
        else:
            plt.plot(cbkrnl.A , color=clrs[z])
        plt.subplot(1,3,3)
        if z==0:
            x_ = cbkrnl.eval(fstats)
        elif z==1:
            y_ = cbkrnl.eval(fstats)
            plt.plot(x_, y_, '.')
plt.show()

plt.plot(fstats.mean(axis=0), 'k')
plt.plot(fobs_z.flatten(), 'ro-')
plt.show()

In [ ]:
for r in range(n_rounds):
    plt.plot(logs[r]['loss'])
    plt.plot(logs_k[r]['loss'])
    plt.plot(logs_k2[r]['loss'])
    plt.legend(['raw', 'x_kl', 'max_ess'])
    plt.show()

In [ ]:
import seaborn


sets = [tds, tds_k, tds_k2]

ess   = np.zeros((len(tds), 3))
mu_   = np.zeros((len(tds), 3, obs_stats.size)) 
sig2_ = np.zeros((len(tds), 3, obs_stats.size))
sig2e = np.zeros((len(tds), 3, obs_stats.size))

for r in range(len(tds)):
    for i in range(3):
        w = sets[i][r][2].reshape(-1,1)
        w /= w.sum()
        ess[r, i] = 1./np.sum(w**2)

        stats = sets[i][r][1]
        sig2e[r, i, :] = np.var( stats, axis = 0)
        mu_[r, i, :]   = np.sum( w * stats,                   axis=0).reshape(1,-1)
        sig2_[r, i, :] = np.sum( w * (stats-mu_[r, i, :])**2, axis=0)    
            
plt.plot(np.arange(1, len(tds)), ess[1:,:])
plt.legend(['raw iws', 'iws + x_kl', 'iws + max_ess', 'unweighted'])
plt.xlabel('round')
plt.ylabel('effective sample size')
plt.show()    

"""
plt.figure(figsize=(12,16))
for i in range(10):
    plt.subplot(5,2,i+1)
    plt.plot(np.arange(0,len(tds)), sig2e[:, :, i], '--')
    plt.plot(np.arange(0,len(tds)), sig2_[:,:,i], linewidth=1.5)    
    plt.legend(['raw', 'x_kl', 'max_ess'])
plt.show()    
"""
plt.figure(figsize=(12,16))
for r in range(1,len(tds)):
    plt.subplot(2,2,r)
    plt.plot(sig2_[r,:,:].T, linewidth=1.5)    
    plt.plot(sig2e[r,1,:].T, 'k--')
    plt.xlabel('# summary statistic')
    plt.ylabel('empirical variance')
    plt.title('SNPE round #' + str(r+1))
    plt.legend(['raw iws', 'iws + x_kl', 'iws + max_ess', 'unweighted'])
plt.show()    


In [ ]:
# run with Gaussian proposals
for r in range(len(tds_k)):
    plot_pdf(posteriors[r],
             pdf2=posteriors_k[r], 
             lims=[-2,2], 
             samples=sam, 
             gt=true_params, 
             resolution=100,
             figsize=(9,9));

In [ ]:
# run with Gaussian proposals
for r in range(len(tds_k)):
    plot_pdf(posteriors[r],
             pdf2=posteriors_k2[r], 
             lims=[-2,2], 
             samples=sam, 
             gt=true_params, 
             resolution=100,
             figsize=(9,9));

In [ ]:
# run with Gaussian proposals
for r in range(len(tds_k)):
    plot_pdf(posteriors[r],
             pdf2=posteriors_k[r], 
             lims=[-2,2], 
             samples=sam, 
             gt=true_params, 
             resolution=100,
             figsize=(9,9));

In [ ]:
# run with Gaussian proposals
for r in range(len(tds_k)):
    plot_pdf(posteriors[r],
             pdf2=posteriors_k2[r], 
             lims=[-2,2], 
             samples=sam, 
             gt=true_params, 
             resolution=100,
             figsize=(9,9));

In [ ]:
# run with Gaussian proposals
for r in range(len(tds_k)):
    plot_pdf(posteriors[r],
             pdf2=posteriors_k[r], 
             lims=[-2,2], 
             samples=sam, 
             gt=true_params, 
             resolution=100,
             figsize=(9,9));

In [ ]:
# run with Gaussian proposals
for r in range(len(tds_k)):
    plot_pdf(posteriors[r],
             pdf2=posteriors_k2[r], 
             lims=[-2,2], 
             samples=sam, 
             gt=true_params, 
             resolution=100,
             figsize=(9,9));

In [ ]:
import theano
import lasagne.layers as ll

network = res.network
inputs  = tds[0][1]
obs_z   = (obs_stats - res.stats_mean) / res.stats_std

ks = list(network.layer.keys())
hiddens = np.where([i[:6]=='hidden' for i in ks])[0]
layer_index = hiddens[0]
hl = network.layer[ks[layer_index]]
print(hl.name)

f_eval = theano.function(
    inputs=[network.stats],
    outputs=ll.get_output(hl))

plt.plot(f_eval(inputs)[:, :].T, 'k.')
plt.plot(f_eval(obs_z).reshape(-1), 'ro')
plt.show()


layer_index = hiddens[-1]
hl = network.layer[ks[layer_index]]
print(hl.name)

f_eval = theano.function(
    inputs=[network.stats],
    outputs=ll.get_output(hl))

plt.plot(f_eval(inputs)[:, :].T, 'k.')
plt.plot(f_eval(obs_z).reshape(-1), 'ro')
plt.show()


print(network.layer['mixture_means'])
f_eval = theano.function(
    inputs=[network.stats],
    outputs=network.dms)
plt.plot(f_eval(inputs)[0].T, 'k.')
plt.plot(f_eval( obs_z)[0].T, 'ro')
plt.plot(true_params, 'bx', ms=5)
plt.show()

print(f_eval( obs_z)[0])
print(posteriors[-1].xs[0].m)
print(true_params)


In [ ]:
import theano
import lasagne.layers as ll

network = res.network
inputs  = tds[-1][1]
obs_z   = (obs_stats - res.stats_mean) / res.stats_std

ks = list(network.layer.keys())
hiddens = np.where([i[:6]=='hidden' for i in ks])[0]
layer_index = hiddens[0]
hl = network.layer[ks[layer_index]]
print(hl.name)

f_eval = theano.function(
    inputs=[network.stats],
    outputs=ll.get_output(hl))

plt.plot(f_eval(inputs)[:, :].T, 'k.')
plt.plot(f_eval(obs_z).reshape(-1), 'ro')
plt.show()


layer_index = hiddens[-1]
hl = network.layer[ks[layer_index]]
print(hl.name)

f_eval = theano.function(
    inputs=[network.stats],
    outputs=ll.get_output(hl))

plt.plot(f_eval(inputs)[:, :].T, 'k.')
plt.plot(f_eval(obs_z).reshape(-1), 'ro')
plt.show()


print(network.layer['mixture_means'])
f_eval = theano.function(
    inputs=[network.stats],
    outputs=network.dms)
plt.plot(f_eval(inputs)[0].T, 'k.')
plt.plot(f_eval( obs_z)[0].T, 'ro')
plt.plot(true_params, 'bx', ms=5)
plt.show()

print(f_eval( obs_z)[0])
print(posteriors[-1].xs[0].m)
print(true_params)

In [ ]:
distm = np.zeros((n_rounds, 3))
dists = np.zeros((n_rounds, 3))

for r in range(n_rounds):

    distm[r,:] = (np.sqrt( np.sum( (posteriors[r].xs[0].m    - sam.mean(axis=1))**2 ) ),
           np.sqrt( np.sum( (posteriors_k[r].xs[0].m  - sam.mean(axis=1))**2 ) ),
           np.sqrt( np.sum( (posteriors_k2[r].xs[0].m - sam.mean(axis=1))**2 ) ))
           
    dists[r,:] = (np.sqrt( np.sum( (np.diag(posteriors[r].xs[0].S) - sam.var(axis=1))**2 ) ),
           np.sqrt( np.sum( (np.diag(posteriors_k[r].xs[0].S) - sam.var(axis=1))**2 ) ), 
           np.sqrt( np.sum( (np.diag(posteriors_k2[r].xs[0].S) - sam.var(axis=1))**2 ) ))        
plt.subplot(1,2,1)
plt.plot(distm[1:,0].T, 'r')
plt.plot(distm[1:,1].T, 'b')
plt.plot(distm[1:,2].T, 'g')
plt.subplot(1,2,2)
plt.plot(dists[1:,0].T, 'r')
plt.plot(dists[1:,1].T, 'b')
plt.plot(dists[1:,2].T, 'g')
plt.show()

In [ ]:
distm = np.zeros((n_rounds, 3))
dists = np.zeros((n_rounds, 3))

for r in range(n_rounds):

    distm[r,:] = (np.sqrt( np.sum( (posteriors[r].xs[0].m    - sam.mean(axis=1))**2 ) ),
           np.sqrt( np.sum( (posteriors_k[r].xs[0].m  - sam.mean(axis=1))**2 ) ),
           np.sqrt( np.sum( (posteriors_k2[r].xs[0].m - sam.mean(axis=1))**2 ) ))
           
    dists[r,:] = (np.sqrt( np.sum( (np.diag(posteriors[r].xs[0].S) - sam.var(axis=1))**2 ) ),
           np.sqrt( np.sum( (np.diag(posteriors_k[r].xs[0].S) - sam.var(axis=1))**2 ) ), 
           np.sqrt( np.sum( (np.diag(posteriors_k2[r].xs[0].S) - sam.var(axis=1))**2 ) ))        
plt.subplot(1,2,1)
plt.plot(distm[1:,0].T, 'r')
plt.plot(distm[1:,1].T, 'b')
plt.plot(distm[1:,2].T, 'g')
plt.subplot(1,2,2)
plt.plot(dists[1:,0].T, 'r')
plt.plot(dists[1:,1].T, 'b')
plt.plot(dists[1:,2].T, 'g')
plt.show()

In [ ]:
distm = np.zeros((n_rounds, 3))
dists = np.zeros((n_rounds, 3))

for r in range(n_rounds):

    distm[r,:] = (np.sqrt( np.sum( (posteriors[r].xs[0].m    - sam.mean(axis=1))**2 ) ),
           np.sqrt( np.sum( (posteriors_k[r].xs[0].m  - sam.mean(axis=1))**2 ) ),
           np.sqrt( np.sum( (posteriors_k2[r].xs[0].m - sam.mean(axis=1))**2 ) ))
           
    dists[r,:] = (np.sqrt( np.sum( (np.diag(posteriors[r].xs[0].S) - sam.var(axis=1))**2 ) ),
           np.sqrt( np.sum( (np.diag(posteriors_k[r].xs[0].S) - sam.var(axis=1))**2 ) ), 
           np.sqrt( np.sum( (np.diag(posteriors_k2[r].xs[0].S) - sam.var(axis=1))**2 ) ))        
plt.subplot(1,2,1)
plt.plot(distm[1:,0].T, 'r')
plt.plot(distm[1:,1].T, 'b')
plt.plot(distm[1:,2].T, 'g')
plt.subplot(1,2,2)
plt.plot(dists[1:,0].T, 'r')
plt.plot(dists[1:,1].T, 'b')
plt.plot(dists[1:,2].T, 'g')
plt.show()

In [ ]:
distm = np.zeros((n_rounds, 3))
dists = np.zeros((n_rounds, 3))

for r in range(n_rounds):

    distm[r,:] = (np.sqrt( np.sum( (posteriors[r].xs[0].m    - sam.mean(axis=1))**2 ) ),
           np.sqrt( np.sum( (posteriors_k[r].xs[0].m  - sam.mean(axis=1))**2 ) ),
           np.sqrt( np.sum( (posteriors_k2[r].xs[0].m - sam.mean(axis=1))**2 ) ))
           
    dists[r,:] = (np.sqrt( np.sum( (np.diag(posteriors[r].xs[0].S) - sam.var(axis=1))**2 ) ),
           np.sqrt( np.sum( (np.diag(posteriors_k[r].xs[0].S) - sam.var(axis=1))**2 ) ), 
           np.sqrt( np.sum( (np.diag(posteriors_k2[r].xs[0].S) - sam.var(axis=1))**2 ) ))        
plt.subplot(1,2,1)
plt.plot(distm[1:,0].T, 'r')
plt.plot(distm[1:,1].T, 'b')
plt.plot(distm[1:,2].T, 'g')
plt.subplot(1,2,2)
plt.plot(dists[1:,0].T, 'r')
plt.plot(dists[1:,1].T, 'b')
plt.plot(dists[1:,2].T, 'g')
plt.show()

In [ ]:
distm = np.zeros((n_rounds, 3))
dists = np.zeros((n_rounds, 3))

for r in range(n_rounds):

    distm[r,:] = (np.sqrt( np.sum( (posteriors[r].xs[0].m    - sam.mean(axis=1))**2 ) ),
           np.sqrt( np.sum( (posteriors_k[r].xs[0].m  - sam.mean(axis=1))**2 ) ),
           np.sqrt( np.sum( (posteriors_k2[r].xs[0].m - sam.mean(axis=1))**2 ) ))
           
    dists[r,:] = (np.sqrt( np.sum( (np.diag(posteriors[r].xs[0].S) - sam.var(axis=1))**2 ) ),
           np.sqrt( np.sum( (np.diag(posteriors_k[r].xs[0].S) - sam.var(axis=1))**2 ) ), 
           np.sqrt( np.sum( (np.diag(posteriors_k2[r].xs[0].S) - sam.var(axis=1))**2 ) ))        
plt.subplot(1,2,1)
plt.plot(distm[1:,0].T, 'r')
plt.plot(distm[1:,1].T, 'b')
plt.plot(distm[1:,2].T, 'g')
plt.subplot(1,2,2)
plt.plot(dists[1:,0].T, 'r')
plt.plot(dists[1:,1].T, 'b')
plt.plot(dists[1:,2].T, 'g')
plt.show()

In [ ]:
distm = np.zeros((n_rounds, 3))
dists = np.zeros((n_rounds, 3))

for r in range(n_rounds):

    distm[r,:] = (np.sqrt( np.sum( (posteriors[r].xs[0].m    - sam.mean(axis=1))**2 ) ),
           np.sqrt( np.sum( (posteriors_k[r].xs[0].m  - sam.mean(axis=1))**2 ) ),
           np.sqrt( np.sum( (posteriors_k2[r].xs[0].m - sam.mean(axis=1))**2 ) ))
           
    dists[r,:] = (np.sqrt( np.sum( (np.diag(posteriors[r].xs[0].S) - sam.var(axis=1))**2 ) ),
           np.sqrt( np.sum( (np.diag(posteriors_k[r].xs[0].S) - sam.var(axis=1))**2 ) ), 
           np.sqrt( np.sum( (np.diag(posteriors_k2[r].xs[0].S) - sam.var(axis=1))**2 ) ))        
plt.subplot(1,2,1)
plt.plot(distm[1:,0].T, 'r')
plt.plot(distm[1:,1].T, 'b')
plt.plot(distm[1:,2].T, 'g')
plt.subplot(1,2,2)
plt.plot(dists[1:,0].T, 'r')
plt.plot(dists[1:,1].T, 'b')
plt.plot(dists[1:,2].T, 'g')
plt.show()

In [ ]:
for r in range(1,n_rounds):
    plt.plot(logs_k2[r]['cbkrnl'].A)

In [ ]:
for r in range(1,n_rounds):
    plt.plot(logs_k[r]['cbkrnl'].A)